In [13]:
import warnings
warnings.filterwarnings("ignore")
import dbutils

main_connector = dbutils.connect_to_main_database()

sql_statement = "SELECT COUNT(*) FROM person;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "persons")

sql_statement = "SELECT COUNT(*) FROM measure;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "measures")

sql_statement = "SELECT COUNT(*) FROM measure WHERE type='manual';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "measures of type manual")
   
sql_statement = "SELECT COUNT(*) FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='rgb';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts of type image")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts of type pointcloud")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE measure_id IS NOT NULL;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts with measure")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE measure_id IS NULL;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts without measure")
    
sql_statement = "SELECT SUM(file_size) FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(int(result / (1024 * 1024 * 1024)), "GBs of data")

sql_statement = "SELECT SUM(file_size) FROM artifact  WHERE type='rgb';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(int(result / (1024 * 1024 * 1024)), "GBs of image data")

sql_statement = "SELECT SUM(file_size) FROM artifact  WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(int(result / (1024 * 1024 * 1024)), "GBs of pointcloud data")

sql_statement = "SELECT COUNT(DISTINCT(qr_code)) FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "distinct QR codes in artifacts")

sql_statement = "SELECT COUNT(DISTINCT(qr_code)) FROM artifact WHERE measure_id IS NULL;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "distinct QR codes in artifacts without measure_id")


6990 persons
13739 measures
7028 measures of type manual
966811 artifacts
743562 artifacts of type image
223249 artifacts of type pointcloud
633915 artifacts with measure
332896 artifacts without measure
378 GBs of data
80 GBs of image data
297 GBs of pointcloud data
5215 distinct QR codes in artifacts
1841 distinct QR codes in artifacts without measure_id


In [2]:
sql_statement = "SELECT artifact.create_date, type FROM artifact ORDER BY artifact.create_date DESC;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "QR codes")

1549605284005 QR codes
